## Defines

In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pickle
import sys
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch import optim
import torch.nn as nn
#from ipdb import set_trace
from datetime import datetime
import time
import copy
from tqdm import tqdm
from ipdb import set_trace
import argparse
from torch.utils.data import Dataset
from matplotlib import image
from matplotlib import pyplot
from torchvision import transforms
from PIL import Image
import imgaug as ia
import imgaug.augmenters as iaa
import random
plt.ion()   

ROOT_DIR               = '../'
NEW_DATASET_DIR        = ROOT_DIR + 'datasets/ntu_dataset/dataset_v3/'


# Util functions

In [ ]:
def onehot(vals, possible_vals):
    if not isinstance(possible_vals, list): raise TypeError("provide possible_vals as a list")
    enc_vals = np.zeros([len(vals), len(possible_vals)])
    for i, value in enumerate(vals):
        if isinstance(possible_vals[0], float):
            enc = np.where(abs(possible_vals-value)<1e-3)
        else:
            enc = np.where(possible_vals==value)
        enc_vals[i,enc] = 1
    return enc_vals

def get_traffic_light_one_hot(traffic_light_state) : 
    if(traffic_light_state[0] == True) : 
        return 0
    else : 
        return 1

class Rescale(object):
    def __init__(self, scalar):
        self.scalar = scalar

    def __call__(self, im):
        w, h = [int(s*self.scalar) for s in im.size]
        return transforms.Resize((h, w))(im)

class Crop(object):
    def __init__(self, box):
        assert len(box) == 4
        self.box = box

    def __call__(self, im):
        return im.crop(self.box)

class Augment(object):
    def __init__(self, seq):
        self.seq = seq

    def __call__(self, im):
        return Image.fromarray(self.seq.augment_images([np.array(im)])[0])

def load_checkpoint(filename) :
    checkpoint = torch.load(filename)
    model_state_dict = checkpoint['model_state_dict']
    optimizer_state_dict = checkpoint['optimizer_state_dict']
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return [model_state_dict, optimizer_state_dict, epoch, loss]

def save_checkpoint(model, optimizer, epoch, loss, filename):
    print("Saving Checkpoint....")
    model_state_dict = model.state_dict()
    optimizer_state_dict = optimizer.state_dict()
    torch.save({
            'epoch': epoch,
            'model_state_dict': model_state_dict,
            'optimizer_state_dict': optimizer_state_dict,
            'loss': loss
            }, filename)
    print("Saving Checkpoint Done")

def display_image(image) : 
    plt.show(image)
    
def display_torch_im(tensor) : 
    print(tensor.size())
    tensor_img_0 = tensor[0].squeeze(0)
    print(tensor_img_0.size())
    plt.imshow(tensor_img_0.permute(1, 2, 0))
    plt.show()
    
def load_checkpoint_weights(checkpoint, model) :
    print("checkpoint name : ", checkpoint)
    _model_state_dict, _optimizer_state_dict, _epoch, _loss = load_checkpoint(checkpoint)
    model.load_state_dict(_model_state_dict)

def resume_model(checkpoint, model, optimizer) : 
    print("checkpoint name : ", checkpoint)
    _model_state_dict, _optimizer_state_dict, _epoch, _loss = load_checkpoint(checkpoint)
    #print(_model_state_dict)
    model.load_state_dict(_model_state_dict)
    optimizer.load_state_dict(_optimizer_state_dict)
    print("resume model succesful")

def plot_tensor_image(tensor_im) :
    plt.imshow(tensor_im.permute(1, 2, 0))
    plt.show()

def get_train_test_split(image_dir_path): 
    image_list = os.listdir(image_dir_path)
    file_name = []
    for file in image_list:
        file_name.append(os.path.splitext(file)[0])
    random.shuffle(file_name) 

    split_1 = int(0.8 * len(file_name))
    train_filenames = file_name[:split_1]
    test_filenames = file_name[split_1:]
    
    return train_filenames, test_filenames


# Model functions

In [ ]:
def train_model(model, dataloaders, dataset_sizes, criterion_dict, criterion_vec, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc       = 0.0
    best_loss      = 0.0
    epoch_acc      = 0.0
    num_outputs    = len(criterion_vec)
    
    train_loss_epoch = []
    val_loss_epoch   = []
     
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, data in enumerate(dataloaders[phase]) :
                inputs_old     = data[0]
                labels         = data[1]
                inputs, labels = convert_inputs(inputs_old ,labels)
                inputs         = inputs.to(device)
                labels         = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward, track history if only in train
                current_batch_size = inputs.size(0)
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    regression_preds, classification_preds = \
                              custom_preds(outputs, current_batch_size, num_outputs)
                    #custom loss for regression + classification head
                    loss, losses    = custom_loss(outputs, labels, criterion_dict, criterion_vec)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        if(i%20 == 0) :
                            print()
                            print_model_output("preds", regression_preds, classification_preds)
                            print_model_label("labels", labels)
                            print("losses : ", losses)
                            print("loss :", loss.item())

                # statistics
                running_loss += loss.item() * inputs.size(0)

                if(phase == 'val') :
                    if(i%10==0) :
                        print_model_output("preds", regression_preds, classification_preds)
                        print_model_label("labels", labels)
                        print("loss :", loss.item())
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            if(phase == 'train') :
                train_loss_epoch.append(epoch_loss)
            elif(phase == 'val') :
                val_loss_epoch.append(epoch_loss)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            #if phase == 'val' and epoch_acc > best_acc:
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'train'  :
                checkpoint_name = ROOT_DIR + 'checkpoints/model_' + str(epoch) + '.tar'
                save_checkpoint(model, optimizer, epoch, epoch_loss, checkpoint_name)
        print("\n*********************************")
        print(train_loss_epoch)
        print(val_loss_epoch)
        print("*********************************\n")
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_loss_epoch, val_loss_epoch

def test_model(model, dataloaders, dataset_sizes, criterion_vec):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 0.0
    model.eval()   # Set model to evaluate mode
    running_corrects = 0
    running_loss = 0.0
    num_outputs    = len(criterion_vec)
    # Iterate over data.
    for i, data in enumerate(dataloaders) :
        inputs = data[0] 
        labels = data[1]
        inputs, labels = convert_inputs(inputs, labels)
        current_batch_size = inputs.size(0)
        if(i%10 == 0) :
            display_torch_im(inputs)
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            regression_preds, classification_preds = \
                                  custom_preds(outputs, current_batch_size, num_outputs)
            #custom loss for regression + classification head
            loss, losses = custom_loss(outputs, labels)
            if(i%10 == 0) :
                print_model_output("preds", regression_preds, classification_preds, 1)
                print_model_label("labels", labels, 1)
                print("losses : ", losses)
                print("loss   : ", loss)
            running_loss += loss.item() * inputs.size(0)
    epoch_loss = 0.0
    epoch_acc  = 0.0
    epoch_loss = running_loss / dataset_sizes
    print('Loss: {:.4f} Acc: {:.4f}'.format(
                epoch_loss, epoch_acc))
    time_elapsed = time.time() - since
    print('Test complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    model.load_state_dict(best_model_wts)
    return model

# Dataset

In [ ]:
def get_augmentations():
    # applies the given augmenter in 50% of all cases,
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image
    seq = iaa.Sequential([
            # execute 0 to 5 of the following (less important) augmenters per image
            iaa.SomeOf((0, 5),
                [
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 3.0)),
                        iaa.AverageBlur(k=(2, 7)),
                        iaa.MedianBlur(k=(3, 11)),
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
                    # search either for all edges or for directed edges,
                    # blend the result with the original image using a blobby mask
                    iaa.SimplexNoiseAlpha(iaa.OneOf([
                        iaa.EdgeDetect(alpha=(0.5, 1.0)),
                        iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                    ])),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                    iaa.OneOf([
                        iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                        iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                    ]),
                    iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                    iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                    # either change the brightness of the whole image (sometimes
                    # per channel) or change the brightness of subareas
                    iaa.OneOf([
                        iaa.Multiply((0.5, 1.5), per_channel=0.5),
                        iaa.FrequencyNoiseAlpha(
                            exponent=(-4, 0),
                            first=iaa.Multiply((0.5, 1.5), per_channel=True),
                            second=iaa.ContrastNormalization((0.5, 2.0))
                        )
                    ]),
                    iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                    sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                ],
                random_order=True
            )
        ],
        random_order=True
    )
    return seq
class Augment(object):
    def __init__(self, seq):
        self.seq = seq

    def __call__(self, im):
        return Image.fromarray(self.seq.augment_images([np.array(im)])[0])

def find_label_max(pickle_data_l, pickle_data_r) : 
    count                = 0
    max_relative_angle   = 0.0
    l_max_centre_dist    = 0.0
    r_max_centre_dist    = 0.0
    for key, value in pickle_data_l.items() :
        if(abs(value['relative_angle']) > max_relative_angle) : 
            max_relative_angle = abs(value['relative_angle'])                                         
        if(abs(value['centre_dist']) > l_max_centre_dist): 
            l_max_centre_dist  = abs(value['centre_dist'])

    for key, value in pickle_data_r.items() :
        if(abs(value['relative_angle']) > max_relative_angle) : 
            max_relative_angle = abs(value['relative_angle'])                                         
        if(abs(value['centre_dist']) > r_max_centre_dist): 
            r_max_centre_dist  = abs(value['centre_dist'])
    print('max relative angle : ', max_relative_angle , " max centre dist (l) : ", l_max_centre_dist, " (r) : ", r_max_centre_dist)
    return max_relative_angle, l_max_centre_dist, r_max_centre_dist

class CARLA_Dataset(Dataset):
    def __init__(self, t, pickle_file_path_l, image_dir_path, image_file_names, pickle_file_path_r):
        #initializing the max range
        self.max_rangle      = 0.0
        self.max_centre_dist = 0.0
        self.max_vdistance   = 50.0
        #initializing transforms
        print("T : ", t)
        assert t in ['train', 'val']
        self.transform = get_data_transforms(t)
        ###############LEFT CAMERA##############
        self.image_path = image_dir_path
        self.image_list = os.listdir(image_dir_path)
        self.file_name = image_file_names
        #initialize the labels 
        #read pickle file
        self.pickle_list_l = os.listdir(pickle_file_path_l)
        self.pickled_data_l = {}
        for file in self.pickle_list_l:
            f = open((pickle_file_path_l + file), 'rb')  
            self.pickled_data_l.update(pickle.load(f))
            f.close() 
        print(self.image_list)
        ###############RIGHT CAMERA##############
        self.pickle_list_r = os.listdir(pickle_file_path_r)
        self.pickled_data_r = {}
        for file in self.pickle_list_r:
            f = open((pickle_file_path_r + file), 'rb')  
            self.pickled_data_r.update(pickle.load(f))
            f.close() 
        f.close() 
        self.max_rangle, self.l_max_centre_dist, self.r_max_centre_dist \
            = find_label_max(self.pickled_data_l, self.pickled_data_r)

    def __len__(self):
        return len(self.file_name)

    def normalize_labels(self, labels_dict, chk_camera) : 
        labels_dict['front_vehicle']  /= self.max_vdistance
        labels_dict['relative_angle'] /= self.max_rangle
        labels_dict['centre_dist']    /= self.r_max_centre_dist
        return labels_dict
    
    def __getitem__(self, idx):
        frames      = []
        inputs = {}  
        labels = {}
        #####################LEFT CAMERA################
        #reading PIL
        image_name = self.file_name[idx] + '.png'
        raw_image = Image.open(os.path.join(self.image_path, image_name)).convert('RGB')
        #reading file name to access the pickle file
        current_fname = self.file_name[idx]
        chk_camera = current_fname[-3:]

        if chk_camera == '_rt' :
            current_fname_r = current_fname[:-3]
            label_dict = self.pickled_data_r[current_fname_r] 
        else :
            label_dict = self.pickled_data_l[current_fname] 
        label_values = list(label_dict.values())
        #transforming regression labels
        label_dict                    = self.normalize_labels(label_dict, chk_camera)
        labels['front_vehicle']       = torch.Tensor(np.array(float(label_dict['front_vehicle'])))
        labels['centre_dist']         = torch.Tensor(np.array(float(label_dict['centre_dist'])))
        labels['pedestrian_distance'] = torch.Tensor(np.array(float(label_dict['pedestrian_distance'])))
        labels_traffic_light          = get_traffic_light_one_hot(label_dict['traffic_light'])
        labels['traffic_light']       = torch.Tensor(np.array(float(labels_traffic_light)))
        labels['relative_angle']      = torch.Tensor(np.array(float(label_dict['relative_angle'])))
        
        im = self.transform(raw_image)
        inputs ['sequence'] = im
        return inputs, labels

#dataset support functions
def get_data_transforms(t='train'):
    data_transforms = {
        'train': transforms.Compose([
#             Augment(get_augmentations()),
            Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
        'val': transforms.Compose([
            Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms[t]

def get_data(batch_size):
    dataset_dir = NEW_DATASET_DIR
    image_dir   = NEW_DATASET_DIR + 'Camera'
    train_filenames, test_filenames = get_train_test_split(image_dir)
    train_ds = CARLA_Dataset( 'train', pickle_file_path_l = dataset_dir+'Left_pickle_file/',
                                       image_dir_path     = dataset_dir+'Camera/',
                                       image_file_names   = train_filenames, 
                                       pickle_file_path_r = dataset_dir+'Right_pickle_file/')
    val_ds   = CARLA_Dataset( 'val',   pickle_file_path_l = dataset_dir+'Left_pickle_file/',
                                       image_dir_path     = dataset_dir+'Camera/',
                                       image_file_names   = test_filenames, 
                                       pickle_file_path_r = dataset_dir+'Right_pickle_file/')
    return (
        len(train_ds),
        len(val_ds),
        DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=1),
        DataLoader(val_ds, batch_size=batch_size*2, num_workers=1),
        train_ds,
        val_ds
    )


In [ ]:
len_train_ds, len_valid_ds, train_dl, valid_dl, train_ds, val_ds = get_data(batch_size=128)
print(len_train_ds)
print(len_valid_ds)

dataloaders   = {'train':train_dl, 'val':valid_dl}
print(dataloaders['train'])
dataset_sizes = {'train':len_train_ds, 'val':len_valid_ds}
print("dataset sizes :", dataset_sizes)

# Model Definitions

## Model for combined for regression and classification prediction

In [ ]:
def custom_preds(output, batch_size, num_outputs) : 
#     print("custom preds : ", output.size(), batch_size, num_outputs)
    num_pred_outputs = num_outputs-1
    num_regression_outputs = 3
    
    regression_preds     = torch.zeros(batch_size, num_regression_outputs)
    classification_preds = torch.zeros(batch_size) 
    
    regression_preds        = output[:,0:3]
    _, classification_preds = torch.max(output[:,3:5], 1) #.float()
    classification_preds    = classification_preds.T.float()
    classification_preds    = classification_preds.unsqueeze(dim=1)
    return regression_preds, classification_preds
    

def custom_loss(output, target, criterion_dict=None, criterion_vec=None) : 
    regression_loss     = 0.0
    classification_loss = 0.0
    loss     = 0.0
    criterion_mse       = nn.MSELoss()
    criterion_bce_logit = nn.BCEWithLogitsLoss()
    criterion_ce        = nn.CrossEntropyLoss()
    losses              = []
    for i in range(3) : 
        task_regression_loss = criterion_mse(output[:,i], target[:,i])
        regression_loss     += task_regression_loss
        losses.append(task_regression_loss.item())
    classification_target = target[:,3].long()
    classification_target = classification_target #.squeeze(dim=1)
    classification_output = output[:,3:5]
    classification_loss = criterion_ce(classification_output, classification_target)
    losses.append(classification_loss.item())
    loss = regression_loss + classification_loss

    return loss, losses

#model definitions (combined regression)
def convert_inputs(inputs_old, labels):
    inputs               = inputs_old['sequence']
    labels_vdistance     = labels['front_vehicle'] #/50.0
    labels_cdistance     = labels['centre_dist']
    labels_direction     = labels['pedestrian_distance']
    labels_rangle        = labels['relative_angle']
    labels_traffic_light = labels['traffic_light']
    labels_traffic_light = labels_traffic_light#.squeeze(dim=1)
    combined_label       = torch.stack([labels_vdistance, labels_cdistance, labels_rangle, labels_traffic_light], dim=1)
    return inputs, combined_label

def print_model_output(name, regression_out, classification_out, num_output=5) : 
    print("****** ", name, " ******")
    print("VD : ", regression_out[0:num_output,0].cpu().detach().numpy(), \
          " \nCD : ", regression_out[0:num_output,1].cpu().detach().numpy(), \
          " \nRA : ", regression_out[0:num_output,2].cpu().detach().numpy(), \
          " \nTL : ", classification_out[0:num_output].T.cpu().detach().numpy())
    
def print_model_label(name, label, num_output=5) :
    print("****** ", name, " ******")
    print("VD : ", label[0:num_output,0].cpu().detach().numpy(), \
          " \nCD : ", label[0:num_output,1].cpu().detach().numpy(), \
          " \nRA : ", label[0:num_output,2].cpu().detach().numpy(), \
          " \nTL : ", label[0:num_output,3].cpu().detach().numpy())


device           = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion_vec    = [1,1,0,0]

print("Choose CNN model (Enter No. ) : ")
print("0. VGG16")
print("1. Resnet18")
print("2. Resnet50")
model_idx = int(input())

if(model_idx < 0 or model_idx > 2) :
    print("Incorrect model selected : ", model_idx, " Expected (0/1/2)")
    
if(model_idx == 0) : 
    print("choosing VGG16....")
    model_ft         = models.vgg16(pretrained=True)
    model_ft.classifier[6].out_features = 5
elif(model_idx == 1) : 
    print("choosing Resnet18....")
    model_ft         = models.resnet18(pretrained=True)
    num_ftrs         = model_ft.fc.in_features
    model_ft.fc      = nn.Linear(num_ftrs, 5)
else : 
    print("choosing Resnet50....")
    model_ft         = models.resnet50(pretrained=True)
    num_ftrs         = model_ft.fc.in_features
    model_ft.fc      = nn.Linear(num_ftrs, 5)
model_ft         = nn.DataParallel(model_ft)
model_ft         = model_ft.to(device)    
optimizer_ft     = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Training

In [ ]:
# # #train model
# print("starting training...")
# # #if(args.retrain == True) : 
# #  #   resume_model(args.checkpoint, model_ft, optimizer_ft)
# resume_model(ROOT_DIR + 'checkpoints/model_14.tar', model_ft, optimizer_ft) 
criterion_dict     = {'MSE' : nn.MSELoss, 'CE' : nn.CrossEntropyLoss}
model_final, train_loss_epoch, val_loss_epoch = train_model(model_ft, dataloaders, dataset_sizes, criterion_dict, criterion_vec, optimizer_ft, exp_lr_scheduler, 15)
print(train_loss_epoch)
print(val_loss_epoch)

dump_name = input("Enter test name")
np.savez('../dumps/'+dump_name+'.npz', train_loss=train_loss_epoch, val_loss_epoch=val_loss_epoch)

# Testing Model

In [ ]:
load_checkpoint_weights(ROOT_DIR +'checkpoints/resnet18_all_affordance/model_16.tar', model_ft)
test_model(model_ft, dataloaders['val'], dataset_sizes['val'], criterion_vec)

# Dataloader unit test

In [ ]:
# #model definitions (combined regression)
# def convert_inputs(inputs_old, labels):
#     inputs               = inputs_old['sequence']
#     labels_vdistance     = labels['front_vehicle']/50
#     labels_cdistance     = labels['centre_dist']
#     labels_direction     = labels['pedestrian_distance']
#     labels_traffic_light = labels['traffic_light']
# #     labels_traffic_light = labels_traffic_light.T
#     #print("driection shape : ", labels_direction.size())
#     print("labels traffic before : ", labels_traffic_light.size())
#     labels_traffic_light = labels_traffic_light.squeeze(dim=1)
#     print("labels traffic after : ", labels_traffic_light.size())
# #     combined_label       = torch.stack([labels_vdistance, labels_cdistance, labels_traffic_light], dim=1)
#     combined_label       = torch.stack([labels_vdistance, labels_cdistance], dim=1)
    
#     print(combined_label)
#     return inputs, combined_label

# train_dataloader                 = dataloaders['train']
# data                             = next(iter(train_dataloader))
# test_input , test_train_label    = data
# converted_input, converted_label = convert_inputs(test_input, test_train_label)

# test_train_im = test_input['sequence']
# print(test_train_label)
# print(test_train_im.size())
# for i in range(16) :
#     print(test_train_label['front_vehicle'][i])
#     print("converted label : ", converted_label[i])
#     plot_tensor_image(test_train_im[i])